<a href="https://colab.research.google.com/github/ishita1508joshi/Indoor-Scene-Classification-Model/blob/main/Indoor_Scene_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import keras
from keras import models
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
import cv2
import time
from google.colab import drive

In [2]:
data_path = '/content/drive/My Drive/Colab Notebooks/indoor_dataset/'

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
categories = ["clothingstore","dining_room","grocerystore","kitchen","library","livingroom","mall","movietheater","museum","restaurant"]

In [4]:
import random
from numpy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
import numpy as np
import os
import cv2

In [5]:
def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

In [6]:
def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

In [7]:
def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels
    return image_array[:, ::-1]

In [8]:
def create_training_data(categories,datadir,img_size,training_data):
    for category in categories:
        path = os.path.join(datadir, category) # path to categories
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) #read image and convert to gray scale
                new_array = cv2.resize(img_array,(img_size,img_size))
                random_rotation(new_array)
                training_data.append([new_array,class_num])
                random_noise(new_array)
                training_data.append([new_array,class_num])
                # horizontal_flip(new_array)
                # training_data.append([new_array,class_num])
            except Exception as e:
                print("Error creating training data")
                pass

In [9]:
img_size = 128     # resize all the images to one size
training_data=[]
create_training_data(categories,data_path,img_size,training_data)
random.shuffle(training_data)
X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)

In [10]:
print(y)

[2, 0, 1, 2, 0, 7, 5, 1, 7, 9, 1, 8, 9, 1, 8, 5, 1, 9, 1, 9, 8, 0, 5, 9, 3, 3, 3, 3, 3, 7, 0, 3, 4, 5, 3, 5, 8, 7, 3, 7, 2, 3, 1, 3, 1, 9, 1, 1, 3, 2, 7, 9, 0, 1, 1, 2, 3, 9, 5, 8, 1, 1, 3, 9, 1, 8, 6, 3, 1, 2, 8, 2, 3, 8, 3, 2, 2, 2, 7, 3, 3, 2, 5, 0, 5, 3, 5, 1, 3, 9, 3, 5, 5, 1, 5, 9, 9, 3, 1, 8, 2, 9, 5, 2, 1, 1, 1, 1, 5, 6, 5, 9, 3, 3, 8, 2, 1, 9, 9, 3, 8, 5, 1, 2, 1, 3, 1, 9, 1, 3, 1, 9, 4, 2, 1, 3, 5, 1, 0, 3, 2, 0, 7, 5, 3, 1, 1, 3, 7, 9, 3, 2, 7, 0, 8, 5, 6, 1, 5, 9, 9, 2, 3, 3, 9, 2, 2, 1, 5, 3, 7, 3, 5, 9, 9, 3, 9, 8, 5, 3, 2, 3, 4, 2, 1, 2, 3, 9, 3, 5, 5, 5, 1, 5, 3, 2, 1, 9, 5, 6, 2, 5, 2, 8, 9, 1, 5, 1, 5, 1, 3, 0, 1, 6, 9, 5, 2, 7, 9, 3, 2, 2, 9, 3, 5, 1, 3, 3, 3, 3, 4, 5, 1, 1, 3, 0, 2, 5, 0, 1, 8, 0, 3, 5, 3, 2, 9, 1, 1, 3, 1, 2, 3, 1, 3, 1, 3, 5, 9, 5, 1, 9, 9, 2, 1, 3, 6, 5, 1, 5, 3, 1, 1, 3, 3, 2, 4, 8, 3, 4, 2, 3, 2, 0, 4, 1, 3, 5, 1, 9, 1, 2, 1, 1, 5, 1, 5, 0, 3, 9, 5, 9, 1, 0, 2, 9, 5, 2, 0, 0, 5, 3, 3, 2, 9, 9, 5, 2, 3, 1, 3, 9, 3, 3, 7, 4, 1, 1, 3, 1, 1, 1, 8, 

In [11]:
import tensorflow
X=np.array(X).reshape(-1,img_size,img_size,1)  #(cannot pass list directly, -1=(calculates the array size), size,1=gray scale)
class_num=tensorflow.keras.utils.to_categorical(y,num_classes=len(categories))   #one-hot encoder for categorical values

In [12]:
X=X/255.0

In [13]:
dense_layers=[4]
layer_sizes=[128]
conv_layers=[2]

In [14]:
print(X[5])
print(X.shape)

[[[0.18039216]
  [0.15294118]
  [0.12941176]
  ...
  [0.14901961]
  [0.10588235]
  [0.11372549]]

 [[0.09019608]
  [0.09803922]
  [0.10588235]
  ...
  [0.10588235]
  [0.01960784]
  [0.05098039]]

 [[0.10980392]
  [0.07843137]
  [0.04313725]
  ...
  [0.02745098]
  [0.12156863]
  [0.0627451 ]]

 ...

 [[0.10196078]
  [0.10588235]
  [0.10980392]
  ...
  [0.23137255]
  [0.13333333]
  [0.11372549]]

 [[0.10980392]
  [0.11372549]
  [0.10588235]
  ...
  [0.27058824]
  [0.11764706]
  [0.11372549]]

 [[0.12156863]
  [0.11764706]
  [0.10588235]
  ...
  [0.23529412]
  [0.10196078]
  [0.10980392]]]
(844, 128, 128, 1)


In [15]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            name = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            print(name)
            tensorboard = TensorBoard(log_dir='C:\Indoor Scene Recognition\logs{}'.format(name))
            model = models.Sequential()
            model.add(Conv2D(layer_size,(3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(0.5))
            model.add(Dense(len(categories), activation='softmax'))
            model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
            model.fit(X, class_num, epochs=10, batch_size=32,validation_split=0.2,callbacks=[tensorboard])
            print('model fit complete')

2-conv-128-nodes-4-dense-1730201113


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 16s 293ms/step - accuracy: 0.1727 - loss: 2.2687 - val_accuracy: 0.2308 - val_loss: 2.0976
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.2003 - loss: 2.0994 - val_accuracy: 0.2308 - val_loss: 2.0512
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.2168 - loss: 2.0289 - val_accuracy: 0.2663 - val_loss: 1.9799
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.3419 - loss: 1.9016 - val_accuracy: 0.2959 - val_loss: 1.9227
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.3937 - loss: 1.7613 - val_accuracy: 0.4379 - val_loss: 1.7756
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.4948 - loss: 1.4812 - val_accuracy: 0.4615 - val_loss: 1.6211
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.6597 - loss: 0.9896 - val_accuracy: 0.6213 - val_loss: 1.2935
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.8190 - loss: 0.5354 - val_accuracy: 0.6805 -

In [16]:
data_path2 = '/content/drive/My Drive/Colab Notebooks/test_dataset/'

In [17]:
def create_testing_data(categories,datadir,img_size,testing_data):
    for category in categories:
        path = os.path.join(datadir, category) # path to categories
        class_num = categories.index(category)

        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) #read image and convert to gray scale
                new_array = cv2.resize(img_array,(img_size,img_size))
                random_rotation(new_array)
                testing_data.append([new_array,class_num])
                random_noise(new_array)
                testing_data.append([new_array,class_num])
                # horizontal_flip(new_array)
                # training_data.append([new_array,class_num])
            except Exception as e:
                print("Error creating testing data")
                pass

In [18]:
img_size = 128     # resize all the images to one size
testing_data=[]
create_testing_data(categories,data_path2,img_size,testing_data)
random.shuffle(testing_data)
X_test = []
y_test = []
for features,label in testing_data:
    X_test.append(features)
    y_test.append(label)

In [19]:
X_test=np.array(X_test).reshape(-1,img_size,img_size,1)  #(cannot pass list directly, -1=(calculates the array size), size,1=gray scale)
class_num=tensorflow.keras.utils.to_categorical(y_test,num_classes=len(categories))   #one-hot encoder for categorical values

In [20]:
X_test = X_test/255.0

In [22]:
prediction = model.predict(X_test)
prediction = [i for sample in prediction for i in range(len(sample)) if sample[i] == max(sample)]

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 974ms/step


In [23]:
print(y_test)
print(prediction)

[2, 8, 4, 7, 2, 8, 9, 6, 3, 4, 2, 3, 3, 2, 5, 1, 0, 0, 0, 8, 3, 9, 7, 6, 4, 7, 0, 8, 1, 1, 9, 6, 6, 5, 5, 4, 1, 9, 5, 7]
[2, 8, 4, 7, 2, 1, 9, 6, 3, 4, 2, 3, 3, 2, 5, 4, 0, 0, 0, 8, 3, 9, 7, 6, 4, 7, 0, 1, 4, 1, 9, 6, 6, 5, 5, 4, 1, 9, 5, 7]


In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score, confusion_matrix
print(accuracy_score(y_test, prediction))
print(precision_score(y_test, prediction, average="weighted"))
print(f1_score(y_test, prediction, average="weighted"))
print(recall_score(y_test, prediction, average="weighted"))
print(confusion_matrix(y_test, prediction))

0.9
0.9166666666666666
0.8966666666666667
0.9
[[4 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 2 0 0 0 0 0]
 [0 0 4 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0]
 [0 0 0 0 0 0 0 4 0 0]
 [0 2 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 4]]
